# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [26]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset


ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.


In [27]:
cluster_name = "compute-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Existing Compute using {cluster_name}")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=6, min_nodes=1)
    compute_target = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [28]:
from azureml.core.dataset import Dataset

ws = Workspace.from_config()
experiment_name = 'capstone-hyperdrive'

experiment=Experiment(ws, experiment_name)
dataset = Dataset.get_by_name(ws, name='salary')

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [29]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig

import os

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.15, delay_evaluation=10)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": choice(0.01, 0.1, 1.0, 10.0),
    "--max_iter": choice(50, 100, 150, 200, 250)
})

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies_hyperdrive.yml')

estimator = ScriptRunConfig(script="train.py",
                        source_directory=".",
                        environment=sklearn_env,
                        compute_target=compute_target)

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                    run_config=estimator,
                                    policy=early_termination_policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_concurrent_runs=4,
                                    max_total_runs=20
                                    )

In [30]:
from azureml.train.hyperdrive import HyperDriveRun
hd_run: HyperDriveRun = experiment.submit(hyperdrive_run_config)
hd_run.wait_for_completion()
# RunDetails(hd_run).show()

{'runId': 'HD_e793d8e4-3216-48a7-9d34-63f256be0250',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-05-08T22:40:44.358133Z',
 'endTimeUtc': '2022-05-08T23:00:52.737967Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4f5e17aa-1aef-4586-96e4-c05f87b0b3d4',
  'user_agent': 'python/3.8.9 (macOS-10.15.7-x86_64-i386-64bit) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.40.0',
  'space_size': '20',
  'score': '0.857362198679564',
  'best_child_run_id': 'HD_e793d8e4-3216-48a7-9d34-63f256be0250_8',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg194966.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_e793d8e4-3216-48a7-9d34-63f256be0250/azureml-logs/hyperdrive.txt

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [32]:
import os
from azureml.core.model import Model
os.makedirs("./outputs", exist_ok=True)
best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_file_names = best_run.get_file_names()
print('Metrics: ', best_run_metrics)
print("\n\nBest run file names :", best_run_file_names)
print('\n\nRun Id: {} Accuracy: {}'.format(best_run.id, best_run_metrics['Accuracy']))
# model_path = "./outputs/{}_model.joblib".format(best_run.id)
final_model: Model = best_run.register_model(model_name = 'hyperdrive_model',model_path = "./outputs/model.joblib")
final_model.download(".")


Metrics:  {'Regularization Strength:': 10.0, 'Max iterations:': 100, 'Accuracy': 0.857362198679564}


Best run file names : ['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'logs/azureml/dataprep/rslex.log.2022-05-08-22', 'outputs/OfflineRun_0c08ae98-773c-4110-859f-110f142d74ac_model.joblib', 'outputs/OfflineRun_4240eb40-857b-4b55-9008-6d8474db0896_model.joblib', 'outputs/OfflineRun_53e84e53-fded-48c8-a7b2-4bd175523904_model.joblib', 'outputs/OfflineRun_710cf2d5-ff0c-47b1-b837-cd38acb6fe32_model.joblib', 'outputs/OfflineRun_7cd92f43-62d1-4556-afba-87d17b182bf6_model.joblib', 'outputs/OfflineRun_87fe93a0-744c-43ee-acc6-4ab8e2b2ede9_model.joblib', 'outputs/OfflineRun_b62c83d5-6571-4e87-83b0-9613fa40a340_model.joblib', 'outputs/OfflineRun_d804e689-26aa-4bdc-b8f5-dcc495c3f796_model.joblib', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hostto

'model.joblib'

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [33]:
from azureml.core.model import InferenceConfig
from azureml.core import Environment
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, LocalWebservice

inference_config = InferenceConfig(
    environment=sklearn_env,
    source_directory=".",
    entry_script="./score.py"
)

# deployment_config = AciWebservice.deploy_configuration(
#     cpu_cores=0.5, memory_gb=1, auth_enabled=True
# )
deployment_config = LocalWebservice.deploy_configuration(port=6789)


service = Model.deploy(
    ws,
    "hdmodelservice",
    [final_model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 7f447ae3207f4d1585cf53c3ec74893d.azurecr.io
Logging into Docker registry 7f447ae3207f4d1585cf53c3ec74893d.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 7f447ae3207f4d1585cf53c3ec74893d.azurecr.io/azureml/azureml_0411c2f901f3867c64dbfc5754860b61
 ---> 97b4d57e5053
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 3b834d9045f4
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjlhNzUxMWI4LTE1MGYtNGE1OC04NTI4LTNlN2Q1MDIxNmMzMSIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE5NDk2NiIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE5NDk2NiIsIndvcmtzcGFjZUlkIjoiN2Y0NDdhZTMtMjA3Zi00ZDE1LTg1Y2YtNTNjM2VjNzQ4OTNkIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 81afc29af179
 ---> 3793e70b64d3
Step 4/5 : RUN mv '/var/azureml-app/tmpezaowms3.py' /var/azureml-app/main.py


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json
from score import process_input
import pandas as pd

# uri = service.scoring_uri
# requests.get(uri)
headers = {"Content-Type": "application/json"}
data = {
    "age": 25,
    "workclass": "Private",
    "fnlwgt": 176756,
    "education": "Masters",
    "education-num": 14,
    "marital-status": "Never-married",
    "occupation": "Sales",
    "sex": "Female",
    "capital-gain": 0,
    "capital-loss": 0,
    "hours-per-week": 45,
    "native-country": "United-States"
}
dft = pd.DataFrame(data)
dft.head()
# data = json.dumps(data)
# response = requests.post(uri, data=data, headers=headers)
# print(response.json())
# print(service.get_logs())

test is {'query': 'What color is the fox', 'context': 'The quick brown fox jumped over the lazy dog.'}
2022-05-07T16:44:02,618171100+00:00 - gunicorn/run 
2022-05-07T16:44:02,618151600+00:00 - iot-server/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-05-07T16:44:02,619041400+00:00 - rsyslog/run 
2022-05-07T16:44:02,624185500+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-05-07T16:44:02,765203700+00:00 - iot-server/finish 1 0
2022-05-07T16:44:02,766644300+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (13)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 43
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2022-05-07 16:44:03,557 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2022-05-07 16:

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

